Сначала импортируем необходимые модули

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import csv

Задаем переменные для ссылок.  
url - основная ссылка   
urlstub - ссылка для прикрепления аттрибутов href.

In [ ]:
urlstub = 'https://www.wdl.org'
url = 'https://www.wdl.org/ru/search/?item_type=print-photograph#1'

Создаем запрос на нашу главную ссылку и делаем из ответа BeautifulSoup.  
Извлекаем из него необходимые для нас ссылки, присваиваем списку ссылок имя titles.

In [ ]:
r = requests.get(url)
bs = BeautifulSoup(r.text, 'html.parser')
titles = bs.select('a.title')

Создаем csv файл с заголовками, куда будем всё сохранять.

In [ ]:
csv_columns = ['Название', 'Время создания', 'Период', 'Место']
with open('pictures.csv', 'a', encoding='utf-8') as pic:
    writer = csv.DictWriter(pic, fieldnames=csv_columns)
    writer.writeheader()

Создаем функцию которая будет проходиться по каждой ссылке и извдекать нужную для нас информацию, записывая в файл pictures.json.  
Между переходами по ссылкам установлена задержка в 0.33 секунды, чтобы не перенагружать сервер запросами.

In [ ]:
def title_parser(title_list):
    for title in titles:
        time.sleep(0.33)

        photolink = urlstub + title.attrs['href']
        r = requests.get(photolink)
        bs = BeautifulSoup(r.text, 'html.parser')

        picture = {}
        picture['Название'] = bs.select_one('h1#page-title').text.strip()
        picture['Время создания'] = bs.select_one('time[itemprop="dateCreated"]').attrs['datetime']
        picture['Период'] = bs.select_one('div#subject-date time').attrs['datetime']
        places = bs.select('div#places ul li')
        picture['Место'] = [place.text.strip().replace('\n', ', ') for place in places]
        csv_columns = ['Название', 'Время создания', 'Период', 'Место']

        with open('pictures.csv', 'a', encoding='utf-8') as pic:
            writer = csv.DictWriter(pic, fieldnames=csv_columns)
            writer.writerow(picture)
            print('Сохранено: ' + picture['Название'])

Запускаем эту функцию с нашим листом ссылок.

In [ ]:
title_parser(titles)

Выводим для пользователя количество сохраненных ссылок.  
Генерируем ссылку для следующей страницы.

In [ ]:
total_saved = len(titles)
print('сохранено ' + str(total_saved) + ' фото')
next_page = 2
next_stub = 'https://www.wdl.org/ru/search/?item_type=print-photograph&page='
next_link = next_stub +str(next_page)

Дальше продолжаем загружать страницы и переходить по ссылкам с фотографиями, извлекая необходимую информацию.
Страницы будут загружаться до тех пор, пока не останется новых страниц.
Если хотим загрузить меньше страниц, то раскомменчиваем фрагмент и указываем нужное число страниц (на одну страницу приходится 42 фото):
if next_page == 6:  
&nbsp;&nbsp;&nbsp;&nbsp;   print('Сохранено пять страниц, хватит')  
&nbsp;&nbsp;&nbsp;&nbsp;   break  
Если этих строчек нет, то будут сохранены данные обо всех фотографиях на примерно 200 страницах.

In [ ]:
while True:
    r = requests.get(next_link)
    if r.status_code != 200:
        print('Следующей страницы не существует')
        break
    bs = BeautifulSoup(r.text, 'html.parser')
    titles = bs.select('a.title')
    title_parser(titles)
    total_saved += len(titles)
    print('сохранено ' + str(total_saved) + ' фото')

    next_page += 1
#     if next_page == 4:
#         print('Сохранено пять страниц, хватит')
#         break
    next_link = next_stub + str(next_page)